In [ ]:
import os
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

def extract_grid_lines(file):
    file = os.path.expanduser(file)  # ~ を展開
    ds = xr.open_dataset(file)
    lat = ds['XLAT_C'].values[0]
    lon = ds['XLONG_C'].values[0]
    ds.close()
    return lat, lon

# ファイル読み込み
lat1, lon1 = extract_grid_lines('~/Run_WRF/example/geo_em.d01.nc')  # 親ドメイン
lat2, lon2 = extract_grid_lines('~/Run_WRF/example/geo_em.d02.nc')  # 子ドメイン

# パラメータ
label_step = 1     # 何本おきにラベルを付けるか
label_size = 10     # ラベル文字サイズ

# 地図描画
fig = plt.figure(figsize=(12, 10))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_title("WRF Nesting Structure with Grid Indices (Parent d01)", fontsize=16)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.LAND, edgecolor='black')
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.LAKES, edgecolor='black')
ax.add_feature(cfeature.RIVERS)

# 親ドメイン（破線）
nj, ni = lat1.shape  # j: 緯度方向(行), i: 経度方向(列)
for j in range(nj):
    ax.plot(lon1[j, :], lat1[j, :], linestyle='--', color='blue', linewidth=0.5, transform=ccrs.PlateCarree())
for i in range(ni):
    ax.plot(lon1[:, i], lat1[:, i], linestyle='--', color='blue', linewidth=0.5, transform=ccrs.PlateCarree())

# ---- 親ドメインの格子線に index ラベルを付与 ----
# j（緯度方向）番号：各 j の「左端」（西端）に表示
for j in range(0, nj, label_step):
    i_west = 0
    xj = lon1[j, i_west]
    yj = lat1[j, i_west]
    ax.text(
        xj, yj, f"j={j+1}",
        fontsize=label_size, color='blue',
        ha='right', va='center',
        transform=ccrs.PlateCarree(),
        zorder=5,
        bbox=dict(boxstyle="round,pad=0.15", fc="white", ec="none", alpha=0.6)
    )

# i（経度方向）番号：各 i の「下端」（南端）に表示
for i in range(0, ni, label_step):
    j_south = 0
    xi = lon1[j_south, i]
    yi = lat1[j_south, i]
    ax.text(
        xi, yi, f"i={i+1}",
        fontsize=label_size, color='blue',
        ha='center', va='top',
        transform=ccrs.PlateCarree(),
        zorder=5,
        bbox=dict(boxstyle="round,pad=0.15", fc="white", ec="none", alpha=0.6)
    )

# 子ドメイン（実線）
for j in range(lat2.shape[0]):
    ax.plot(lon2[j, :], lat2[j, :], linestyle='-', color='red', linewidth=0.5, transform=ccrs.PlateCarree())
for i in range(lat2.shape[1]):
    ax.plot(lon2[:, i], lat2[:, i], linestyle='-', color='red', linewidth=0.5, transform=ccrs.PlateCarree())


# === 中心緯線・中心経線を描く（CEN_LAT / CEN_LON） ===

cen_lat = ds.attrs['CEN_LAT']
cen_lon = ds.attrs['CEN_LON']

# いまの地図の表示範囲（PlateCarree座標）を取得
xmin, xmax, ymin, ymax = ax.get_extent(crs=ccrs.PlateCarree())

# 緯線（y=一定）：左端→右端へ
ax.plot([xmin, xmax], [cen_lat, cen_lat],
        linestyle='-', linewidth=1.2, transform=ccrs.PlateCarree(),
        zorder=6)

# 経線（x=一定）：下端→上端へ
ax.plot([cen_lon, cen_lon], [ymin, ymax],
        linestyle='-', linewidth=1.2, transform=ccrs.PlateCarree(),
        zorder=6)

# 交点にマーカー
ax.scatter([cen_lon], [cen_lat], transform=ccrs.PlateCarree(),
           s=25, zorder=7)

# ラベル（右端に緯線ラベル、上端に経線ラベル）
ax.text(xmax, cen_lat, f"ref_lat",
        ha='right', va='bottom', fontsize=12,
        transform=ccrs.PlateCarree(),
        bbox=dict(boxstyle="round,pad=0.15", fc="white", ec="none", alpha=0.7),
        zorder=7)

ax.text(cen_lon, ymax, f"ref_lon",
        ha='left', va='top', fontsize=12,
        transform=ccrs.PlateCarree(),
        bbox=dict(boxstyle="round,pad=0.15", fc="white", ec="none", alpha=0.7),
        zorder=7)

plt.tight_layout()
plt.show()
